In [5]:
from functions import download_youtube
#video_id = 'ek05M8eCk7M'
video_id = 'I-cxigjLG0I'
youtube_url = f"https://www.youtube.com/watch?v={video_id}"
job_id = 'Hanni'
file_path, file_name =  download_youtube(youtube_url, job_id=job_id)
file_name

[youtube] Extracting URL: https://www.youtube.com/watch?v=I-cxigjLG0I
[youtube] I-cxigjLG0I: Downloading webpage
[youtube] I-cxigjLG0I: Downloading ios player API JSON
[youtube] I-cxigjLG0I: Downloading player e38bb6de


         n = eFmJDdxcy9AWGsKI ; player = https://www.youtube.com/s/player/e38bb6de/player_ias.vflset/en_US/base.js
         n = RcZkLRWTBK-pvMse ; player = https://www.youtube.com/s/player/e38bb6de/player_ias.vflset/en_US/base.js


[youtube] I-cxigjLG0I: Downloading m3u8 information
[info] Testing format 614
[info] I-cxigjLG0I: Downloading 1 format(s): 614+140
[hlsnative] Downloading m3u8 manifest
[hlsnative] Total fragments: 26
[download] Destination: Hanni_I-cxigjLG0I.f614.mp4
[download] 100% of   17.37MiB in 00:00:01 at 13.30MiB/s                  
[download] Destination: Hanni_I-cxigjLG0I.f140.m4a
[download] 100% of    2.09MiB in 00:00:00 at 2.78MiB/s   
[Merger] Merging formats into "Hanni_I-cxigjLG0I.mp4"
Deleting original file Hanni_I-cxigjLG0I.f614.mp4 (pass -k to keep)
Deleting original file Hanni_I-cxigjLG0I.f140.m4a (pass -k to keep)


'Hanni_I-cxigjLG0I.mp4'

In [6]:
from functions import slice_video
slice_video(file_name, [(0, 240, 'hanni2.mp4')], height=540, width=960)

2024-09-13 10:20:37,355 - INFO - Successfully created segment: hanni2.mp4


[{'output_name': 'hanni2.mp4',
  'start': 0,
  'duration': 240,
  'success': True,
  'error': None}]

In [10]:
import cv2
import face_recognition

# 특정 인물의 이미지를 로드하여 인코딩
known_image = face_recognition.load_image_file("test_hanni2.jpg")
known_face_encoding = face_recognition.face_encodings(known_image)[0]

# 비디오 파일 로드
video_capture = cv2.VideoCapture('hanni.mp4')

# 변수 초기화
trackers = []
face_names = []
face_distances = []
frame_skip = 12
frame_count = 0

while True:
    # 비디오에서 한 프레임씩 읽기
    ret, frame = video_capture.read()

    if not ret:
        break

    frame_count += 1

    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    if frame_count % frame_skip == 0:
        # 추적기와 얼굴 정보 초기화
        trackers = []
        face_names = []
        face_distances = []

        # 얼굴 위치와 인코딩 탐지
        face_locations = face_recognition.face_locations(rgb_frame)
        face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

        for face_location, face_encoding in zip(face_locations, face_encodings):
            top, right, bottom, left = face_location

            # 얼굴 거리 계산
            distance = face_recognition.face_distance([known_face_encoding], face_encoding)[0]

            # 임계값 설정 (예: 0.6)
            tolerance = 0.46

            if distance < tolerance:
                name = "Specific Person"
            else:
                name = "Unknown"

            # 추적기 초기화
            tracker = cv2.legacy.TrackerKCF_create()
            bbox = (left, top, right - left, bottom - top)
            tracker.init(frame, bbox)
            trackers.append(tracker)
            face_names.append(name)
            face_distances.append(distance)
    else:
        # 모든 추적기 업데이트
        new_trackers = []
        new_face_names = []
        new_face_distances = []

        for tracker, name, distance in zip(trackers, face_names, face_distances):
            success, bbox = tracker.update(frame)
            if success:
                new_trackers.append(tracker)
                new_face_names.append(name)
                new_face_distances.append(distance)
                left, top, width, height = [int(v) for v in bbox]
                right = left + width
                bottom = top + height

                # 사각형과 텍스트 표시
                if name == "Specific Person":
                    # 특정 인물에 대해 빨간색 사각형과 이름 및 거리 표시
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)
                    # 이름 표시
                    cv2.putText(frame, name, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                    # 거리 표시 (이름 아래)
                    cv2.putText(frame, f"Distance: {distance:.2f}", (left, bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)
                else:
                    # 다른 인물에 대해 초록색 사각형과 이름만 표시
                    cv2.rectangle(frame, (left, top), (right, bottom), (0, 255, 0), 2)
                    cv2.putText(frame, name, (left, bottom + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    # 거리 표시 (이름 아래)
                    cv2.putText(frame, f"Distance: {distance:.2f}", (left, bottom + 45), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)

        # 추적기와 얼굴 정보 업데이트
        trackers = new_trackers
        face_names = new_face_names
        face_distances = new_face_distances

    # 비디오 출력
    cv2.imshow('Video', frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 모든 작업 완료 후 클린업
video_capture.release()
cv2.destroyAllWindows()
